In [14]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models

print(torch.__version__)
# 检查CUDA是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

2.0.1+cu118
cuda
Files already downloaded and verified
Files already downloaded and verified


In [15]:
vgg = models.vgg16(pretrained=True)
num_ftrs = vgg.classifier[6].in_features
vgg.classifier[6] = nn.Linear(num_ftrs, 10)  # 适应 CIFAR-10 的 10 个类别

/usr/local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /output/.torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:40<00:00, 13.5MB/s]  


In [16]:
vgg = vgg.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg.parameters(), lr=0.001, momentum=0.9)

In [17]:
for epoch in range(10):
    running_loss = 0.0
    total_batches = 0
    correct_train = 0
    total_train = 0
    
    vgg.train()  # 确保模型处于训练模式
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        # 将输入和标签转移到 GPU
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = vgg(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        total_batches += 1
        
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
    
    average_loss = running_loss / total_batches
    train_accuracy = 100 * correct_train / total_train
    print(f'Epoch {epoch + 1}: Average Loss {average_loss:.3f}, Training Accuracy {train_accuracy:.2f}%')

Epoch 1: Average Loss 0.570, Training Accuracy 80.17%
Epoch 2: Average Loss 0.291, Training Accuracy 89.93%
Epoch 3: Average Loss 0.213, Training Accuracy 92.61%
Epoch 4: Average Loss 0.161, Training Accuracy 94.42%
Epoch 5: Average Loss 0.122, Training Accuracy 95.76%
Epoch 6: Average Loss 0.093, Training Accuracy 96.76%
Epoch 7: Average Loss 0.070, Training Accuracy 97.56%
Epoch 8: Average Loss 0.057, Training Accuracy 97.96%
Epoch 9: Average Loss 0.042, Training Accuracy 98.58%
Epoch 10: Average Loss 0.034, Training Accuracy 98.84%


In [18]:
# 在整个测试数据集上计算准确性
correct = 0
total = 0
vgg.eval()  # 确保模型处于评估模式
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # 将输入和标签转移到 GPU
        images, labels = images.to(device), labels.to(device)
        
        outputs = vgg(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Accuracy on test set: {test_accuracy:.2f}%')

Accuracy on test set: 92.53%
